# Implementing double buffer handling in  `main.c`

In this task section we will write the code to manage the stream of samples from the ADC, as well as the interrupt routines which are triggered by the ADC. The goal is to implement the "buffer switching" stage in [this animation](Figures/DualBufferAnim.gif) which leads up to the RFFT calculation.

### a)
The very first thing we need to do is define a window length for the RFFT. In this assignment we will be using window size $N=1024$, so use this as a basis for answering the theory questions later. You are welcome to experiment with other window sizes later, but there are two details wich will need to be taken into account: the window size must be a power of 2 due to how the FFT algorithm works, and the available clock speed/memory on the microprocesser sets an upper limit to possible window sizes.<br><br>
The recommended approach do defining the window length is to create a macro using `#define`:


>```C
>/* USER CODE BEGIN PD */
>#define WINDOW_SIZE 1024  // MUST be power of 2
>/* USER CODE END PD */
>```

We will also need to define an ADC buffer size, wich should be a function of window size $N$. Given our desire to implement a double buffer, how large should the ADC buffer be?


In [2]:
ADC_BUFFER_SIZE = "???"
### BEGIN SOLUTION
WINDOW_SIZE = 1024
ADC_BUFFER_SIZE = 2*WINDOW_SIZE
### END SOLUTION

In [4]:
from hashlib import sha1
assert sha1(str(round(float(ADC_BUFFER_SIZE), 0)).encode('utf-8')+b'c76c6').hexdigest() == '223290ae0be17cf9ca2239d896059f3e2e20bca2', 'Wrong answer for ADC_BUFFER_SIZE :('
print('Correct answer for ADC_BUFFER_SIZE :)')
### BEGIN HIDDEN TESTS
import numpy as np
import hashlib
import random

variable_name = 'ADC_BUFFER_SIZE'
solution = globals()[variable_name]
task_ID = 0x3a
precision = 0

solution = float(solution)
random.seed(task_ID)
salt = hex(int(hashlib.sha256((str(random.randint(0, 0xFF))).encode('utf-8')).hexdigest(), 16) % 10**6)[2:]
test_result = hashlib.sha1(str(round(solution, precision)).encode('utf-8')+bytes(salt, "utf-8")).hexdigest()
test_string =f"from hashlib import sha1\nassert sha1(str(round(float({variable_name}), {precision})).encode('utf-8')+b'{salt}').hexdigest() == '{test_result}', 'Wrong answer for {variable_name} :('\nprint('Correct answer for {variable_name} :)')"
print(test_string)
### END HIDDEN TESTS

Correct answer for ADC_BUFFER_SIZE :)
from hashlib import sha1
assert sha1(str(round(float(ADC_BUFFER_SIZE), 0)).encode('utf-8')+b'c76c6').hexdigest() == '223290ae0be17cf9ca2239d896059f3e2e20bca2', 'Wrong answer for ADC_BUFFER_SIZE :('
print('Correct answer for ADC_BUFFER_SIZE :)')


## b)
Once we have defined the buffer size, we need to create the ADC buffer which takes the form of a regular array with length `ADC_BUFFER_SIZE`. Given our configuration of the ADC in task 2, how many bits should we use for each integer element in the buffer? (In other words, should we use `uint8_t`, `uint16_t`, `uint32_t`, `uint64_t` etc..)


In [7]:
n_bits = "???" # Integer value
### BEGIN SOLUTION
n_bits = 16
### END SOLUTION

In [9]:
from hashlib import sha1
assert sha1(str(round(float(n_bits), 0)).encode('utf-8')+b'cc656').hexdigest() == '2565b1792a2e64d586c5915c6d2a095e5d09636c', 'Wrong answer for n_bits :('
print('Correct answer for n_bits :)')
### BEGIN HIDDEN TESTS
import numpy as np
import hashlib
import random

variable_name = 'n_bits'
solution = globals()[variable_name]
task_ID = 0x3b
precision = 0

solution = float(solution)
random.seed(task_ID)
salt = hex(int(hashlib.sha256((str(random.randint(0, 0xFF))).encode('utf-8')).hexdigest(), 16) % 10**6)[2:]
test_result = hashlib.sha1(str(round(solution, precision)).encode('utf-8')+bytes(salt, "utf-8")).hexdigest()
test_string =f"from hashlib import sha1\nassert sha1(str(round(float({variable_name}), {precision})).encode('utf-8')+b'{salt}').hexdigest() == '{test_result}', 'Wrong answer for {variable_name} :('\nprint('Correct answer for {variable_name} :)')"
print(test_string)
### END HIDDEN TESTS

Correct answer for n_bits :)
from hashlib import sha1
assert sha1(str(round(float(n_bits), 0)).encode('utf-8')+b'cc656').hexdigest() == '2565b1792a2e64d586c5915c6d2a095e5d09636c', 'Wrong answer for n_bits :('
print('Correct answer for n_bits :)')


Once you have landed on a data type, you can declare the buffer in the `USER CODE PV` section of `main.c` using the following template and replacing `/* Your_type_here! */` with a suitable data type:

>```C
>/* USER CODE BEGIN PV */
>// Buffer management
>/* Your_type_here! */ ADC_buffer[ADC_BUFFER_SIZE]; // Buffer to fill with values from ADC.
>__IO uint8_t buffer_full = 0;       // Flag used by interrupt handlers to signal when buffer is ready.
>uint32_t read_buffer_start = 0;     // Index of first sample in RFFT window
>/* USER CODE END PV */
>```

*PS: The `__IO` tag on `buffer_full` marks it as a [volatile](https://en.wikipedia.org/wiki/Volatile_(computer_programming)) variable, which is generally desirable for variables wich run the risk of being accessed by both interrupt routines and the main program "simultaneously".*

## c)

The final piece of the puzzle is to describe our interrupt callback routines. We will ned two separate ones for our frequency detection system:
* `HAL_ADC_ConvCpltCallback` is called once the buffer has been filled, and the ADC begins writing to the start of the buffer again.
* `HAL_ADC_ConvHalfCpltCallback` is called once the buffer is filled to it's halfway point.

Each of these interrupt routines should do 2 things:
1. Raise a flag to signal to the main loop that `WINDOW_SIZE` new samples are ready for processing.
2. Communicate to the main loop ***which*** $N=$`WINDOW_SIZE` number of values within `ADC_buffer` to read from. This can be done by communicating the index value of the first of our RFFT input segment (i.e. a value of `0` would indicate the first half of `ADC_Buffer`). For clarification, see the [animated illustration](Figures/DualBufferAnim.gif).

Below is the suggested code for the interrupt routines, with some crucial information removed:

>```C
>/* USER CODE BEGIN PFP */
>void HAL_ADC_ConvCpltCallback(ADC_HandleTypeDef* hadc)
>{
>	read_buffer_start = /* startpoint_a */ ;
>	buffer_full = 1; // Signals that WINDOW_SIZE new samples are ready for the RFFT
>}
>
>void HAL_ADC_ConvHalfCpltCallback(ADC_HandleTypeDef* hadc)
>{
>	read_buffer_start = /* startpoint_b */ ;
>	buffer_full = 1; // Signals that WINDOW_SIZE new samples are ready for the RFFT
>}
>/* USER CODE END PFP */
>```

What values should we add to the program in place of `startpoint_a` and `startpoint_b`?




In [13]:
startpoint_a = "???"
startpoint_b = "???"
### BEGIN SOLUTION
startpoint_a = WINDOW_SIZE
startpoint_b = 0
### END SOLUTION

In [14]:
from hashlib import sha1
assert sha1(str(round(float(startpoint_a), 0)).encode('utf-8')+b'cc656').hexdigest() == 'bb42e981bd71e8fbbb5b0186cef7614fc98299cd', 'Wrong answer for startpoint_a :('
print('Correct answer for startpoint_a :)')
assert sha1(str(round(float(startpoint_b), 0)).encode('utf-8')+b'cc656').hexdigest() == '802b3ac05e13da5af515a08d1dd4a5c8a1d89cbf', 'Wrong answer for startpoint_b :('
print('Correct answer for startpoint_b :)')
### BEGIN HIDDEN TESTS
import numpy as np
import hashlib
import random


task_ID = 0x3b
precision = 0
variable_name = 'startpoint_a'
solution = globals()[variable_name]

solution = float(solution)
random.seed(task_ID)
salt = hex(int(hashlib.sha256((str(random.randint(0, 0xFF))).encode('utf-8')).hexdigest(), 16) % 10**6)[2:]
test_result = hashlib.sha1(str(round(solution, precision)).encode('utf-8')+bytes(salt, "utf-8")).hexdigest()
test_string =f"from hashlib import sha1\nassert sha1(str(round(float({variable_name}), {precision})).encode('utf-8')+b'{salt}').hexdigest() == '{test_result}', 'Wrong answer for {variable_name} :('\nprint('Correct answer for {variable_name} :)')"
print(test_string)

variable_name = 'startpoint_b'
solution = globals()[variable_name]

solution = float(solution)
random.seed(task_ID)
salt = hex(int(hashlib.sha256((str(random.randint(0, 0xFF))).encode('utf-8')).hexdigest(), 16) % 10**6)[2:]
test_result = hashlib.sha1(str(round(solution, precision)).encode('utf-8')+bytes(salt, "utf-8")).hexdigest()
test_string =f"assert sha1(str(round(float({variable_name}), {precision})).encode('utf-8')+b'{salt}').hexdigest() == '{test_result}', 'Wrong answer for {variable_name} :('\nprint('Correct answer for {variable_name} :)')"
print(test_string)
### END HIDDEN TESTS

Correct answer for startpoint_a :)
Correct answer for startpoint_b :)
from hashlib import sha1
assert sha1(str(round(float(startpoint_a), 0)).encode('utf-8')+b'cc656').hexdigest() == 'bb42e981bd71e8fbbb5b0186cef7614fc98299cd', 'Wrong answer for startpoint_a :('
print('Correct answer for startpoint_a :)')
assert sha1(str(round(float(startpoint_b), 0)).encode('utf-8')+b'cc656').hexdigest() == '802b3ac05e13da5af515a08d1dd4a5c8a1d89cbf', 'Wrong answer for startpoint_b :('
print('Correct answer for startpoint_b :)')


## e)
Once all the pieces are in place, we are ready to add the code which will start up the sampling process:

```C
/* USER CODE BEGIN 2 */
HAL_Delay(100); /* Add a short delay between ADC config and ADC startup.*/
HAL_TIM_PWM_Start(&htim2, TIM_CHANNEL_1);
HAL_ADC_Start_DMA(&hadc1, /* Start sampling on ADC instance hadc1 */
                  (uint32_t*) ADC_buffer, /* Cast ADC_Buffer as an array of uint32_t values */
                  ADC_BUFFER_SIZE);
/* USER CODE END 2 */
```

Add the code and verify that the project is compiled successfully. You can now do some testing with the debugger/HAL_UART_Transmit if you wish to verify that the buffer setup works as intended, or you can move on to problem 4: RFFT implementation.